# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4950, uptime 0:00:34
mini_beamline                    RUNNING   pid 4951, uptime 0:00:34
random_walk                      RUNNING   pid 4952, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4953, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4954, uptime 0:00:34
simple                           RUNNING   pid 4955, uptime 0:00:34
thermo_sim                       RUNNING   pid 4956, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4957, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 16:11:17
Persistent Unique Scan ID: '9635585b-2bc5-4a3c-9055-63dd4e55901e'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 16:11:17.4 |     -1.000 |      93740 |


|         2 | 16:11:17.5 |     -0.500 |      94412 |
|         3 | 16:11:17.5 |      0.000 |      95184 |
|         4 | 16:11:17.5 |      0.500 |      94667 |
|         5 | 16:11:17.5 |      1.000 |      93024 |
+-----------+------------+------------+------------+
generator scan ['9635585b'] (scan num: 3)


('9635585b-2bc5-4a3c-9055-63dd4e55901e',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/9635585b-2bc5-4a3c-9055-63dd4e55901e-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 16:11:17
Persistent Unique Scan ID: 'd0a19f05-8f1c-4a91-aac5-e0fbba758c6d'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 16:11:17.9 |          -1 | 1056998394 |


|         2 | 16:11:18.1 |          -0 | 1061536579 |


|         3 | 16:11:18.2 |           0 | 1070710768 |


|         4 | 16:11:18.3 |           0 | 1077296629 |


|         5 | 16:11:18.4 |           1 | 1083127321 |
+-----------+------------+-------------+------------+
generator scan ['d0a19f05'] (scan num: 4)





('d0a19f05-8f1c-4a91-aac5-e0fbba758c6d',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 16:11:18
Persistent Unique Scan ID: 'e4c5cbd5-ef07-4112-9607-cecb37cbd1aa'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 16:11:18.9 | 1107871108 |


|         2 | 16:11:19.8 | 1127588237 |


|         3 | 16:11:20.8 | 1074530295 |


...

|        28 | 16:11:45.9 | 1058793984 |


|        29 | 16:11:46.9 | 1112826620 |


|        30 | 16:11:47.9 | 1126029393 |


+-----------+------------+------------+
generator count ['e4c5cbd5'] (scan num: 5)





('e4c5cbd5-ef07-4112-9607-cecb37cbd1aa',)